In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime,timedelta

import bokeh.palettes as bp
from bokeh.plotting import figure,curdoc, show, output_notebook
output_notebook()
from bokeh.transform import linear_cmap
from bokeh.layouts import column, row
from bokeh.models import (CDSView, 
                          HoverTool,ColorBar,
                          GeoJSONDataSource, 
                          Patches,
                          RadioButtonGroup,
                          DateSlider,
                          Button)

Loading BokehJS ...

In [2]:
# ====================================================================
# Goal: Visualize demographics and daily new cases statistics in Switzerland

# The visualization output contains 2 parts: 

# Part 1: add a map view and the color encodes the Density and BedsPerCapita, 
# with a RadioButtonGroup which controls which aspect to show. 

# Part 2: add a circle in each canton on the map, size of circle represents the current daily new cases per capita,
# add a timeline slider, add callback function controlling the slider and changing the size of the circle on the map when dragging the time slider.
# Additionally, a "Play" button can animate the slider as well as circles on the map.  

# Reference link
# https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

# ====================================================================

In [3]:
### Task 1: Data Preprocessing

## T1.1 Read and filter data 
# Four data sources:
# Demographics.csv: the statistics data about population density and hospital beds per capita in each canton
# covid_19_cases_switzerland_standard_format.csv: the location(longitude, latitude) of the capital city in each canton
# covid19_cases_switzerland_openzh-phase2.csv: same as in ex2, daily new cases in each canton
# gadm36_CHE_1.shp: the shape file contains geometry data of swiss cantons, and is provided in the "data" folder. 
# Please do not submit any data file with your solution, and you can asssume your solution is at the same directory as data 

In [4]:
demo_url = 'https://raw.githubusercontent.com/daenuprobst//covid19-cases-switzerland/master/demographics.csv'
local_url = 'https://raw.githubusercontent.com/daenuprobst//covid19-cases-switzerland/master/covid_19_cases_switzerland_standard_format.csv'
case_url = 'https://raw.githubusercontent.com/daenuprobst//covid19-cases-switzerland/master/covid19_cases_switzerland_openzh-phase2.csv'
shape_dir = 'data/gadm36_CHE_1.shp'

In [5]:
# Read from demo_url into a dataframe using pandas
demo_raw = pd.read_csv(demo_url)

In [6]:
demo_raw.head()

,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita
0,ZH,1520968,37796.0,377.96,4024.150704,0.170,258565,4472,0.002940
1,BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950
2,VD,799145,29940.0,299.40,2669.154977,0.164,131060,2268,0.002838
3,AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138
4,SG,507697,19408.0,194.08,2615.916117,0.183,92909,1565,0.003083


In [7]:
# Read from local_url into a dataframe using pandas
local_raw = pd.read_csv(local_url)

In [8]:
local_raw.head()

,date,country,abbreviation_canton,name_canton,lat,long,hospitalized_with_symptoms,intensive_care,total_hospitalized,home_confinment,total_currently_positive_cases,new_positive_cases,recovered,deaths,total_positive_cases,tests_performed
0,2020-03-06T00:00:00,Switzerland,AG,Aargau,47.449406,8.327495,NaN,NaN,NaN,NaN,10.0,0.0,NaN,NaN,NaN,NaN
1,2020-03-06T00:00:00,Switzerland,AI,Appenzell Innerrhoden,47.328414,9.409647,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-03-06T00:00:00,Switzerland,AR,Appenzell Ausserrhoden,47.382710,9.271860,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN
3,2020-03-06T00:00:00,Switzerland,BE,Bern,46.916667,7.466667,NaN,NaN,NaN,NaN,20.0,0.0,NaN,0.0,NaN,NaN
4,2020-03-06T00:00:00,Switzerland,BL,Baselland,47.482779,7.742975,NaN,NaN,NaN,NaN,9.0,0.0,NaN,NaN,NaN,NaN


In [9]:
# Extract unique 'abbreviation_canton','lat','long' combinations from local_raw
canton_point = local_raw[['abbreviation_canton','lat','long']].drop_duplicates()

In [10]:
canton_point

,abbreviation_canton,lat,long
0,AG,47.449406,8.327495
1,AI,47.328414,9.409647
2,AR,47.382710,9.271860
3,BE,46.916667,7.466667
4,BL,47.482779,7.742975
5,BS,47.558395,7.573271
6,FR,46.718391,7.074008
7,GE,46.195602,6.148113
8,GL,47.040570,9.068036
9,GR,46.796756,10.305946


In [11]:
# Read from case_url into a dataframe using pandas
case_raw = pd.read_csv(case_url)

In [12]:
# case_raw['Date'] = pd.to_datetime(case_raw['Date'], format='%Y-%m-%d')

In [13]:
# Create a date list from case_raw and convert to datatime form
dates = list(case_raw["Date"])
dates

['2020-05-31',
 '2020-06-01',
 '2020-06-02',
 '2020-06-03',
 '2020-06-04',
 '2020-06-05',
 '2020-06-06',
 '2020-06-07',
 '2020-06-08',
 '2020-06-09',
 '2020-06-10',
 '2020-06-11',
 '2020-06-12',
 '2020-06-13',
 '2020-06-14',
 '2020-06-15',
 '2020-06-16',
 '2020-06-17',
 '2020-06-18',
 '2020-06-19',
 '2020-06-20',
 '2020-06-21',
 '2020-06-22',
 '2020-06-23',
 '2020-06-24',
 '2020-06-25',
 '2020-06-26',
 '2020-06-27',
 '2020-06-28',
 '2020-06-29',
 '2020-06-30',
 '2020-07-01',
 '2020-07-02',
 '2020-07-03',
 '2020-07-04',
 '2020-07-05',
 '2020-07-06',
 '2020-07-07',
 '2020-07-08',
 '2020-07-09',
 '2020-07-10',
 '2020-07-11',
 '2020-07-12',
 '2020-07-13',
 '2020-07-14',
 '2020-07-15',
 '2020-07-16',
 '2020-07-17',
 '2020-07-18',
 '2020-07-19',
 '2020-07-20',
 '2020-07-21',
 '2020-07-22',
 '2020-07-23',
 '2020-07-24',
 '2020-07-25',
 '2020-07-26',
 '2020-07-27',
 '2020-07-28',
 '2020-07-29',
 '2020-07-30',
 '2020-07-31',
 '2020-08-01',
 '2020-08-02',
 '2020-08-03',
 '2020-08-04',
 '2020-08-

In [14]:
# Read shape file from shape_dir unsing geopandas
shape_raw = gpd.read_file("./data/gadm36_CHE_1.shp")

In [15]:
shape_raw.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,CHE,Switzerland,CHE.1_1,Aargau,Argovia|ArgÂ¢via|Argovie,None,Canton|Kanton|Chantun,Canton,None,CH.AG,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ..."
1,CHE,Switzerland,CHE.2_1,Appenzell Ausserrhoden,Appenzell Ausser-Rhoden|Appenzell Outer Rhodes...,None,Canton|Kanton|Chantun,Canton,None,CH.AR,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ..."
2,CHE,Switzerland,CHE.3_1,Appenzell Innerrhoden,Appenzell Inner-Rhoden|Appenzell Inner Rhodes|...,None,Canton|Kanton|Chantun,Canton,None,CH.AI,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4..."
3,CHE,Switzerland,CHE.4_1,Basel-Landschaft,BÃ¢le-Campagne|Basel-Country|Baselland|Basel-L...,None,Canton|Kanton|Chantun,Canton,None,CH.BL,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4..."
4,CHE,Switzerland,CHE.5_1,Basel-Stadt,BÃ¢le-Ville|Basel-City|Basel-Town|Basilea-Cita...,None,Canton|Kanton|Chantun,Canton,None,CH.BS,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ..."


In [16]:
# Extract canton name abbreviations from the attribute 'HASC_1', e.g. CH.AG --> AG, CH.ZH --> ZH
# And save into a new column named 'Canton' 
shape_raw['Canton'] = list(map(lambda x: x[3:5], list(shape_raw["HASC_1"])))
canton_poly = shape_raw[['geometry','Canton']]

In [17]:
canton_poly.head()

,geometry,Canton
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS


In [18]:
canton_point

,abbreviation_canton,lat,long
0,AG,47.449406,8.327495
1,AI,47.328414,9.409647
2,AR,47.382710,9.271860
3,BE,46.916667,7.466667
4,BL,47.482779,7.742975
5,BS,47.558395,7.573271
6,FR,46.718391,7.074008
7,GE,46.195602,6.148113
8,GL,47.040570,9.068036
9,GR,46.796756,10.305946


In [19]:
demo_raw.head()

,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita
0,ZH,1520968,37796.0,377.96,4024.150704,0.170,258565,4472,0.002940
1,BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950
2,VD,799145,29940.0,299.40,2669.154977,0.164,131060,2268,0.002838
3,AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138
4,SG,507697,19408.0,194.08,2615.916117,0.183,92909,1565,0.003083


In [20]:
## T1.2 Merge data and build a GeoJSONDataSource

# Merge canton_poly with demo_raw on attribute name 'Canton' into dataframe merged,
# then merge the result with canton_point on 'Canton' and 'abbreviation_canton' respectively

In [21]:
# Potential issue: 
# https://stackoverflow.com/questions/57045479/is-there-a-way-to-fix-maximum-recursion-level-in-python-3
merged=pd.merge(canton_poly, demo_raw, on="Canton")
merged=pd.merge(merged, canton_point, how="left", left_on="Canton", right_on="abbreviation_canton")
merged.head()

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,abbreviation_canton,lat,long
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138,AG,47.449406,8.327495
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766,AR,47.382710,9.271860
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115,AI,47.328414,9.409647
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020,BL,47.482779,7.742975
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156,BS,47.558395,7.573271


In [22]:
desired_columns=['AG_diff_pc', 
                 'AI_diff_pc',
                 'AR_diff_pc',
                 'BE_diff_pc',
                 'BL_diff_pc',
                 'BS_diff_pc',
                 'FR_diff_pc',
                 'GE_diff_pc',
                 'GL_diff_pc',
                 'GR_diff_pc',
                 'JU_diff_pc',
                 'LU_diff_pc',
                 'NE_diff_pc',
                 'NW_diff_pc',
                 'OW_diff_pc',
                 'SG_diff_pc',
                 'SH_diff_pc',
                 'SO_diff_pc',
                 'SZ_diff_pc',
                 'TG_diff_pc',
                 'TI_diff_pc',
                 'UR_diff_pc',
                 'VD_diff_pc',
                 'VS_diff_pc',
                 'ZG_diff_pc',
                 'ZH_diff_pc']

In [23]:
# For each date, extract a list of daily new cases per capita from all cantons(e.g. 'AG_diff_pc', 'AI_diff_pc', etc.), and add as a new column in merged
# For instance, in the column['2020-10-31'], there are: [0.0005411327220155498, nan, nan, 0.000496300306803826, ...]
for i,d in enumerate(dates):
    merged.loc[:,d]=pd.Series(case_raw[case_raw["Date"]==d][desired_columns].values[0])

In [24]:
merged

,geometry,Canton,Population,SettlementAreaHa,SettlementAreaKm2,Density,O65,O65P,Beds,BedsPerCapita,...,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08
0,"POLYGON ((7.94951 47.27751, 7.94897 47.27542, ...",AG,678207,23854.0,238.54,2843.158380,0.177,120043,1450,0.002138,...,0.000180,0.000472,0.000596,0.000537,0.000553,0.000559,0.000495,0.000180,0.000460,NaN
1,"POLYGON ((9.20708 47.27694, 9.20828 47.27905, ...",AR,55234,2231.0,22.31,2475.750784,0.197,10881,208,0.003766,...,NaN,0.000557,0.000557,0.000186,0.000124,0.000248,NaN,NaN,0.000619,NaN
2,"MULTIPOLYGON (((9.51250 47.40100, 9.51430 47.4...",AI,16145,813.0,8.13,1985.854859,0.191,3084,18,0.001115,...,NaN,0.001032,0.000869,0.000724,NaN,0.000960,NaN,NaN,0.000760,0.001068
3,"MULTIPOLYGON (((7.36526 47.41939, 7.36377 47.4...",BL,288132,9025.0,90.25,3192.598338,0.219,63101,582,0.002020,...,0.000303,0.000256,0.000291,0.000521,0.000411,0.000401,0.000531,0.000401,0.000211,0.000433
4,"POLYGON ((7.55254 47.54676, 7.55290 47.54829, ...",BS,194766,2628.0,26.28,7411.187215,0.199,38758,1199,0.006156,...,0.000403,0.000292,0.000545,0.000604,0.000458,0.000545,0.000566,0.000288,0.000458,0.000722
5,"MULTIPOLYGON (((7.09284 46.89419, 7.09202 46.8...",BE,1034977,41197.0,411.97,2512.263029,0.208,215275,3053,0.002950,...,0.000472,0.000288,0.000431,0.000508,0.000637,0.000431,0.000616,0.000385,0.000277,0.000483
6,"MULTIPOLYGON (((6.78575 46.73603, 6.78559 46.7...",FR,318714,13998.0,139.98,2276.853836,0.157,50038,547,0.001716,...,0.000157,0.000323,0.000424,0.000477,0.000395,0.000361,0.000317,0.000213,0.000270,NaN
7,"MULTIPOLYGON (((6.23440 46.33147, 6.23259 46.3...",GE,495249,9416.0,94.16,5259.653781,0.164,81221,1506,0.003041,...,0.000127,0.000402,0.000406,0.000325,0.000357,0.000299,0.000157,0.000105,0.000394,NaN
8,"POLYGON ((8.90771 46.80860, 8.90603 46.81009, ...",GL,40403,1995.0,19.95,2025.213033,0.201,8121,86,0.002129,...,NaN,0.000545,0.000248,0.000347,0.000520,0.000743,NaN,NaN,0.000743,0.000718
9,"MULTIPOLYGON (((10.22766 46.61207, 10.22734 46...",GR,198379,13863.0,138.63,1430.996177,0.213,42255,546,0.002752,...,0.000287,0.000565,0.000565,0.000504,0.000514,0.000620,0.000343,0.000212,0.000489,NaN


In [25]:
# Calculate circle sizes that are proportional to dnc per capita
# Set the latest dnc as default 
merged['size'] = merged.iloc[:,-1]*1e5/5+10
merged['dnc'] = merged.iloc[:,-2]

In [26]:
# Build a GeoJSONDataSource from merged
geosource = GeoJSONDataSource(geojson=merged.to_json())

In [29]:
# Task 2: Data Visualization
def bkapp(doc):
    # T2.1 Create linear color mappers for 2 attributes in demo_raw: population density, hospital beds per capita 
    # Map their maximum values to the high, and mimimum to the low
    labels = ['Density','BedsPerCapita']

    mappers = {}
    mappers['Density'] = linear_cmap("Density", palette=bp.Magma256, low=min(demo_raw["Density"]), high=max(demo_raw["Density"]))
    mappers['BedsPerCapita'] = linear_cmap("BedsPerCapita", palette=bp.Magma256, low=min(demo_raw["BedsPerCapita"]), high=max(demo_raw["BedsPerCapita"]))

    # T2.2 Draw a Switzerland Map on canton level

    # Define a figure 
    p1 = figure(title = 'Demographics in Switzerland', 
                plot_height = 600 ,
                plot_width = 950, 
                toolbar_location = 'above',
                tools = "pan, wheel_zoom, box_zoom, reset")

    p1.xgrid.grid_line_color = None
    p1.ygrid.grid_line_color = None
    # Plot the map using patches, set the fill_color as mappers['Density']
    cantons = p1.patches(xs="xs", ys="ys", fill_color=mappers["Density"], source=geosource)
    # Create a colorbar with mappers['Density'] and add it to above figure
    color_bar = ColorBar(color_mapper=mappers["Density"]["transform"], title="Density", location=(0,0))
    p1.add_layout(color_bar, 'right')
    # Add a hovertool to display canton, density, bedspercapita and dnc 
    hover = HoverTool(tooltips=[("Canton", "@Canton"),
                                ("Population Density", "@Density"+"{int}"),
                                ("Beds Per Capita", "@BedsPerCapita"),
                                ("Daily New Cases per Capita", "@dnc")] ,renderers=[cantons])
    p1.add_tools(hover)
    # T2.3 Add circles at the locations of capital cities for each canton, and the sizes are proportional to daily new cases per capita
    sites = p1.circle(x="long", y="lat", size="size", source=geosource)
    # T2.4 Create a radio button group with labels 'Density', and 'BedsPerCapita'
    buttons = RadioButtonGroup(labels=labels, active=0)
    # Define a function to update color mapper used in both patches and colorbar 
    def update_bar(new):
        for i,d in enumerate(labels):
            if i == new:
                color_bar.color_mapper=list(mappers.items())[new][1]["transform"]
                color_bar.title=list(mappers.items())[new][0]
                cantons = p1.patches(xs="xs", ys="ys", fill_color=list(mappers.items())[new][1], source=geosource)
                sites = p1.circle(x="long", y="lat", size="size", source=geosource)
                p1.add_layout(color_bar, "right")                
    buttons.on_click(update_bar)
    
    # T2.5 Add a dateslider to control which per capita daily new cases information to display
    # Define a dateslider using maximum and mimimum dates, set value to be the latest date
    timeslider = DateSlider(name='Date:', start=pd.to_datetime(dates[0]), end=pd.to_datetime(dates[len(dates)-1]), value=pd.to_datetime(dates[len(dates)-1]))

    # Complete the callback function 
    # Hints: 
        #convert the timestamp value from the slider to datetime and format it in the form of '%Y-%m-%d'
        #update columns 'size', 'dnc' with the column named '%Y-%m-%d' in merged
        #update geosource with new merged
    
    def callback(attr,old,new):
        # Convert timestamp to datetime
        # https://stackoverflow.com/questions/9744775/how-to-convert-integer-timestamp-to-python-datetime
        date = datetime.fromtimestamp(new / 1e3)
        i = date.strftime('%Y-%m-%d')

        merged["size"] = merged.loc[:,i]*1e5/5+10
        merged["dnc"] = merged.loc[:,i]
        geosource.geojson = merged.to_json()

    # Circles change on mouse move
    timeslider.on_change("value", callback)
    
    # T2.6 Add a play button to change slider value and update the map plot dynamically
    # https://stackoverflow.com/questions/46420606/python-bokeh-add-a-play-button-to-a-slider
    # https://stackoverflow.com/questions/441147/how-to-subtract-a-day-from-a-date

    # Update the slider value with one day before current date
    def animate_update_slider():
        # Extract date from slider's current value 
        date = datetime.fromtimestamp(timeslider.value / 1e3)
        #Subtract one day from date and do not exceed the allowed date range
        #I made it go back to the start if it does exceed
        day = date - timedelta(days=1)
        if day<pd.to_datetime(dates[0]):
            day=pd.to_datetime(dates[len(dates)-1])
        timeslider.value = day

    # Define the callback function of button
    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = doc.add_periodic_callback(animate_update_slider, 500)
        else:
            button.label = '► Play'
            doc.remove_periodic_callback(callback_id)

    button = Button(label='► Play', width=80, height=40)
    button.on_click(animate)

    lt = column(p1,buttons, row(timeslider,button))
    doc.add_root(lt)
show(bkapp)